In [1]:
import sys
import math
import time
import string
import pickle
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

from models import miLSTM, LN_miLSTM, LSTM
from utils import list_iter, char2tensor

from sklearn.utils import shuffle

In [2]:
n_characters = len(string.printable)
alphabet = string.printable
n_epochs = 3
print_every = 500
plot_every = 100
hidden_size = 1024

decoder = miLSTM(n_characters, hidden_size, 64, n_characters)
decoder.load_state_dict(torch.load('../models/milstm_1024.pth'))
decoder.cuda()


miLSTM (
  (encoder): Embedding(100, 64)
  (weight_fh): Linear (1024 -> 1024)
  (weight_ih): Linear (1024 -> 1024)
  (weight_zh): Linear (1024 -> 1024)
  (weight_oh): Linear (1024 -> 1024)
  (weight_fx): Linear (64 -> 1024)
  (weight_ix): Linear (64 -> 1024)
  (weight_zx): Linear (64 -> 1024)
  (weight_ox): Linear (64 -> 1024)
  (decoder): Linear (1024 -> 100)
)

In [3]:
def evaluate(prime_str='A', predict_len=100, temperature=0.8):
    decoder.eval()

    hx, cx = decoder.init_hidden()
    prime_input = char2tensor(prime_str)
    predicted = prime_str

    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hx, cx = decoder(prime_input[p], hx, cx)
    inp = prime_input[-1]

    for p in range(predict_len):
        output, hx, cx = decoder(inp, hx, cx)

        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]

        # Add predicted character to string and use as next input
        predicted_char = alphabet[top_i]
        predicted += predicted_char
        inp = char2tensor(predicted_char)

    return predicted.split('<EOS>')[0]

In [7]:
uppers = string.ascii_uppercase

for _ in range(500):
    rand_char = uppers[np.random.permutation(len(uppers))[0]]
    print(evaluate(rand_char, 100), '\n')

Gutted Feast 

Melech 

Blood Prest 

You 

Morbid Defilement 

Parasitical Mandality 

Enslavement 

Merculvera 

Terrorable Life 

Corpse Grave 

Reaction 

Invasion 

Urible Hate 

Genital Putrefied 

Unborn 

Yarstergae 

Hope's Extreme 

Jarax 

Asarhaddon 

Kings of Death 

All Infinite Season 

Eisenfact 

Gaiasa 

Eisening 

Victims of Suffering 

Quetal Infection 

Zombification 

Human Stench 

Detriment of Invagination 

Jeffering 

Necrotic Dying Silence 

Devolving Mortis 

Mortal Decay 

Scum 

Youthcorpse 

Azarith 

Rathergoat 

Grind Hate 

Empty Strangulation 

Death Aeon 

Index 

Detriment 

Cadaveric Mentence 

Warfaie 

Last 

Quetus 

Whorecollectomy 

Logic Soulchres 

Necropsy 

Execration 

Image to Command 

Yaras 

Descend to Ruin 

Yatanas 

Medicine 

Yarven 

Psykhyn 

Catchwet 

Hebhos 

Vakedeath 

Hlavanage 

Whoretot 

Insane 

Grim 

Indigor 

Xenocide 

Recrucement 

Aggression 

Gate of Here 

Istendie 

Infinite Christ 

Lamentation 

Kill Creatio